<a href="https://colab.research.google.com/github/sruthi1996/DeepLearning/blob/master/multiclassClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import  train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
import tensorflow as tf

In [0]:
!pip install PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id':"1uFe4KwMpX4YPnc2H4z4zxepF7RJxm65M"})
download.GetContentFile('seeds.csv')

In [0]:
df = pd.read_csv('seeds.csv')

In [0]:
df.head(5)

,Area,Perimeter,Compactness,Kernel.Length,Kernel.Width,Asymmetry.Coeff,Kernel.Groove,Type
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [0]:
df.isnull().sum()

Area               0
Perimeter          0
Compactness        0
Kernel.Length      0
Kernel.Width       0
Asymmetry.Coeff    0
Kernel.Groove      0
Type               0
dtype: int64

In [0]:
output_pred = df["Type"]
input_layers = df.drop(['Type'],axis=1)
input_layers

,Area,Perimeter,Compactness,Kernel.Length,Kernel.Width,Asymmetry.Coeff,Kernel.Groove
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175
...,...,...,...,...,...,...,...
194,12.19,13.20,0.8783,5.137,2.981,3.631,4.870
195,11.23,12.88,0.8511,5.140,2.795,4.325,5.003
196,13.20,13.66,0.8883,5.236,3.232,8.315,5.056
197,11.84,13.21,0.8521,5.175,2.836,3.598,5.044


In [0]:
output_pred

0      1
1      1
2      1
3      1
4      1
      ..
194    3
195    3
196    3
197    3
198    3
Name: Type, Length: 199, dtype: int64

In [0]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoding = LabelEncoder()
encoding.fit(output_pred)
encode_y = encoding.transform(output_pred)
df_ylabel = np_utils.to_categorical(encode_y)
print(df_ylabel)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0.

In [0]:
def model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=4, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
estimator = KerasClassifier(build_fn=model, epochs=100, batch_size=5, verbose=0)

In [0]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True)

In [0]:
#from sklearn.model_selection import cross_val_score
#results = cross_val_score(estimator, input_layers,df_ylabel , cv=kfold)
#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: nan% (nan%)


In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(input_layers,df_ylabel,test_size=.2,random_state=0)

In [0]:
from keras.models import Sequential
from keras.layers import Dense,Dropout, Flatten
from keras.optimizers import Adam,sgd
from sklearn.metrics import mean_absolute_error
from keras import layers, metrics
import keras

In [0]:
def runAlgo(X_train,X_test,y_train,y_test):
    logi = Sequential()
    logi.add(Dense(10, activation="relu",input_shape=(7,)))
    logi.add(Dropout(0.0001))
    logi.add(Dense(3,activation='softmax'))
    logi.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    logi.fit(X_train, y_train, epochs=400)
    a = logi.evaluate(X_test, y_test)
    return a

In [0]:
runAlgo(X_train,X_test,y_train,y_test)

Epoch 1/400
159/159 [==============================] - 0s 450us/step - loss: 6.4108 - accuracy: 0.3648
Epoch 2/400
159/159 [==============================] - 0s 50us/step - loss: 5.9970 - accuracy: 0.3648
Epoch 3/400
159/159 [==============================] - 0s 49us/step - loss: 5.7601 - accuracy: 0.3836
Epoch 4/400
159/159 [==============================] - 0s 49us/step - loss: 5.5639 - accuracy: 0.6226
Epoch 5/400
159/159 [==============================] - 0s 48us/step - loss: 5.3519 - accuracy: 0.6352
Epoch 6/400
159/159 [==============================] - 0s 51us/step - loss: 5.1286 - accuracy: 0.5094
Epoch 7/400
159/159 [==============================] - 0s 52us/step - loss: 4.9405 - accuracy: 0.3962
Epoch 8/400
159/159 [==============================] - 0s 54us/step - loss: 4.7405 - accuracy: 0.3774
Epoch 9/400
159/159 [==============================] - 0s 53us/step - loss: 4.5498 - accuracy: 0.3774
Epoch 10/400
159/159 [==============================] - 0s 56us/step - loss: 4.35

[0.3757752776145935, 0.8500000238418579]